## Machine Learning

Machine learning is a subset of artificial intelligence in the field of computer science that often uses statistical techniques to give computers the ability to "learn" (i.e., progressively improve performance on a specific task) with data, without being explicitly programmed.


### Supervised Learning

Classification: predict if an input image is a cat or dog.

Regression: predict house pricing base on input features

<img src="https://cdn-images-1.medium.com/max/1600/0*WE3Sz--1NUEWBmUR." alt="classification-regression" style="width: 500px;"/>

### Unsupervised Learning

Unsupervised machine learning is the machine learning task of inferring a function that describes the structure of "unlabeled" data. 

eg. Clustering Task

<img src="https://cdn-images-1.medium.com/max/1600/1*93DYqjjbniZFa7L9JRVLOQ.png" alt="classification-regression" style="width: 500px;"/>

### Our Task

[Kaggle](https://kaggle.com) example - [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic)

Before diving in, we need to equip us with the correct tools!

### Pandas - powerful Python data analysis toolkit

Pandas is a Python package providing fast, flexible, and expressive data structures designed to make working with “relational” or “labeled” data both easy and intuitive. It aims to be the fundamental high-level building block for doing practical, real world data analysis in Python.

http://pandas.pydata.org/pandas-docs/stable/index.html

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("pandas version: ", pd.__version__)

In [ ]:
# First let's create a DataFrame

dates = pd.date_range('20170101', periods=6)
dates

In [ ]:
df = pd.DataFrame(np.random.randn(6,4),index=dates,columns=list('ABCD'))
df

In [ ]:
df2 = pd.DataFrame({ 'A' : 1.,
                     'B' : pd.Timestamp('20130102'),
                     'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                     'D' : np.array([3] * 4,dtype='int32'),
                     'E' : pd.Categorical(["test","train","test","train"]),
                     'F' : 'foo' })
df2

In [ ]:
df2.dtypes

In [ ]:
### show available "column names" and "public attributes"

# df2.<Press Tab>

In [ ]:
df.head()

In [ ]:
df.tail(3)

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
# describe a quick statistic summary
df.describe()

In [ ]:
# transpose
df.T

In [ ]:
df.T.describe()

In [ ]:
df

In [ ]:
# sorting by an axis, availability: 0 or 1
df.sort_index(axis=0, ascending=False)

In [ ]:
# sorting by values
df.sort_values('B')

### Selection

In [ ]:
df['A']

In [ ]:
df[0:3]

In [ ]:
df['20170102':'20170104']

In [ ]:
# By Label
df.loc[dates[0]]

In [ ]:
# By position
df.iloc[3:5, 0:2]

#### Boolean Indexing
Using a single column's values to select data

In [ ]:
df[df.A > 0]

In [ ]:
df.A

In [ ]:
df[df > 0]

In [ ]:
# isin()

df2 = df.copy()
df2['E'] = ['one', 'one', 'two', 'three', 'four', 'three']
df2

In [ ]:
df2[df2['E'].isin(['two','four'])]

In [ ]:
s1 = pd.Series([1,2,3,4,5,6], index=pd.date_range('20170102', periods=6))
s1

In [ ]:
df['F'] = s1
df

In [ ]:
df.at[dates[0],'A'] = 0
df

In [ ]:
df.iat[0,1] = 0
df

### Missing Data

pandas primarily uses the value np.nan to represent missing data. It is by default not included in computations. 

Reindexing allows you to change/add/delete the index on a specified axis. This returns a copy of the data.


In [ ]:
df1 = df.reindex(index=dates[0:4], columns=list(df.columns) + ['E'])
df1.loc[dates[0]:dates[1],'E'] = 1
df1

In [ ]:
df1.dropna(how='any') # how='all'

In [ ]:
df3 = df1.fillna(value=5)
df3

In [ ]:
df1

In [ ]:
pd.isna(df1)

### Stats

In [ ]:
# Stats
df.mean()

In [ ]:
# on other axis
df.mean(1)

### The Titanic Example

![titanic-example](https://img.buzzfeed.com/buzzfeed-static/static/2018-05/22/19/asset/buzzfeed-prod-web-03/anigif_sub-buzz-4981-1527032798-1.gif?downsize=715:*&output-format=auto&output-quality=auto)

[**Important** concepts](https://medium.com/technology-nineleaps/some-key-machine-learning-definitions-b524eb6cb48)

#### Feature

Features are individual independent variables that act as the input in your system. Prediction models use features to make predictions. New features can also be obtained from old features using a method known as ‘feature engineering’. More simply, you can consider one column of your data set to be one feature. Sometimes these are also called attributes. And the number of features are called dimensions.

#### Target

The target is whatever the output of the input variables. It could be the individual classes that the input variables maybe mapped to in case of a classification problem or the output value range in a regression problem. If the training set is considered then the target is the training output values that will be considered.

#### Label

Labels are the final output. You can also consider the output classes to be the labels. When data scientists speak of labeled data, they mean groups of samples that have been tagged to one or more labels.

#### Train

While training for machine learning, you pass an algorithm with training data. The learning algorithm finds patterns in the training data such that the input parameters correspond to the target. The output of the training process is a machine learning model which you can then use to make predictions. This process is also called “learning”.

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
from sklearn.linear_model import LogisticRegression

%matplotlib inline

In [ ]:
train = pd.read_csv("./titanic/train.csv")
test = pd.read_csv("./titanic/test.csv")

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
test.head() # which column is missing?

In [ ]:
## Join train and test datasets in order to obtain the same number of features during categorical conversion
train_len = len(train)
dataset = pd.concat(objs=[train, test], axis=0).reset_index(drop=True) # Axis is important

print("Train data length =>", len(train))
print("Test data length =>", len(test))
print("Concat dataset length =>", len(dataset))

In [ ]:
# Check for Null values
dataset.isnull().sum()

#### Feature Visualization

Correlation matrix between numerical values (SibSp Parch Age and Fare values) and Survived 

In [ ]:
g = sns.heatmap(train[["Survived","SibSp","Parch","Age","Fare"]].corr(),annot=True, fmt = ".2f", cmap = "coolwarm")

Seems `Fare` has a strong positive correlation with `Survived`.

Here the question comes:
1. How many people in train has `fare` greater than 50?
2. What is the percentage?

#### Explore SibSp feature vs Survived

In [ ]:
g = sns.factorplot(x="SibSp",y="Survived",data=train,kind="bar", size = 6 , 
palette = "muted")
g.despine(left=True)
g = g.set_ylabels("survival probability")

**Sex vs Survived**

In [ ]:
g = sns.barplot(x="Sex",y="Survived",data=train)
g = g.set_ylabel("Survival Probability")

Repeat this process for all features, after the process you should have a big picture of how the final output - `Survived` is possibly affected by each feature.

#### Feature Engineer

In [ ]:
# Get Title from Name
dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
dataset["Title"] = pd.Series(dataset_title)
dataset["Title"].unique()

In [ ]:
g = sns.countplot(x="Title",data=dataset)
g = plt.setp(g.get_xticklabels(), rotation=45) 

In [ ]:
# Classify different titles into different priority group 
dataset["Title"] = dataset["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
dataset["Title"] = dataset["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
dataset["Title"] = dataset["Title"].astype(int)

In [ ]:
g = sns.countplot(dataset["Title"])
g = g.set_xticklabels(["Master","Miss/Ms/Mme/Mlle/Mrs","Mr","Rare"])

In [ ]:
g = sns.factorplot(x="Title",y="Survived",data=dataset,kind="bar")
g = g.set_xticklabels(["Master","Miss-Mrs","Mr","Rare"])
g = g.set_ylabels("survival probability")

For each interested feature, find a way to group/simplify/scale it so the model can better understand it mathematically.

In [ ]:
dataset.head()

We will skip some parts and directly load the fully processed data by

In [ ]:
dataset = pd.read_csv('./titanic/data-processed.csv')

In [ ]:
dataset.head()

#### Modeling

**Model**: A machine learning model can be a mathematical representation of a real-world process. To generate a machine learning model you will need to provide training data to a machine learning algorithm to learn from.

**Logistic Regression**: it measures the relationship between the dependent variable (our label, what we want to predict) and the one or more independent variables (our features), by estimating probabilities using it’s underlying logistic function.

![Logistic-Regression](http://www.cntk.ai/jup/cntk103b_TwoFormsOfLR-v3.png)
<img src="https://cdn-images-1.medium.com/max/1600/0*0DnQislGAq1j3uSK.png" alt="LR2" style="width: 500px;"/>

**Sigmoid Function**: A sigmoid function is a mathematical function having a characteristic "S"-shaped curve or sigmoid curve. Often, sigmoid function refers to the special case of the logistic function shown in the first figure and defined by the formula.

**K-fold Validation (optional)**: The goal of cross-validation is to test the model’s ability to predict new data that were not used in estimating it, in order to flag problems like overfitting and to give an insight on how the model will generalize to an independent dataset (i.e., an unknown dataset, for instance from a real problem).

In [ ]:
## Separate train dataset and test dataset
train_len = 881 # hard code 

train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=["Survived"],axis = 1,inplace=True)

In [ ]:
## Separate train features and label 

train["Survived"] = train["Survived"].astype(int)
Y_train = train["Survived"]
X_train = train.drop(labels = ["Survived"],axis = 1)

In [ ]:
# Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [ ]:
random_state = 2
classifier = LogisticRegression(random_state = random_state)
scores = cross_val_score(classifier, X_train, y = Y_train, scoring = "accuracy", cv = kfold, n_jobs=4)

In [ ]:
scores

#### Predict

In [ ]:
classifier = classifier.fit(X_train, Y_train)
prediction = pd.Series(classifier.predict(test), name="Survived")

#### Write Results

In [ ]:
test = pd.read_csv("./titanic/test.csv")
test

In [ ]:
results = pd.concat([test["PassengerId"],prediction],axis=1)
# results.to_csv("prediction.csv",index=False)
results

#### Read List
[Logistic Regression](https://towardsdatascience.com/the-logistic-regression-algorithm-75fe48e21cfa)

#### Code Credit:
https://www.kaggle.com/leyuanyu/titanic-top-4-with-ensemble-modeling

#### Image Source
*Classification vs Regression*: https://cdn-images-1.medium.com/max/1600/0*WE3Sz--1NUEWBmUR.

*Unsupervised Learning*: https://cdn-images-1.medium.com/max/1600/1*93DYqjjbniZFa7L9JRVLOQ.png

*Logistic Regression: http://www.cntk.ai/jup/cntk103b_TwoFormsOfLR-v3.png*

*Sigmoid Function*: https://cdn-images-1.medium.com/max/1600/0*0DnQislGAq1j3uSK.png